<div align="center">
  <h1 align="center">LizardByte Developer Dashboard</h1>
</div>

<div align="center">
[
  <a href="#Developer-Tools">Developer Tools</a> •
  <a href="#Repository-Data">Repository Data</a> •
  <a href="#Star-Gazers">Star Gazers</a> •
  <a href="#Forks">Forks</a> •
  <a href="#Open-Issues">Open Issues</a> •
  <a href="#Open-PRs">Open PRs</a> •
  <a href="#License-Distribution">License Distribution</a> •
  <a href="#Coverage">Coverage</a> •
  <a href="#Programming-Languages">Programming Languages</a> •
  <a href="#Documentation">Documentation</a>
]
</div>

<div align="center" id="Developer-Tools">
  <h2>Developer Tools</h2>
  <a href="https://app.codecov.io/gh/LizardByte">
    <img src="https://img.shields.io/badge/codecov-button?style=for-the-badge&logo=codecov&color=gray" alt="CodeCov">
  </a>
  <a href="https://crowdin.com/project/lizardbyte">
    <img src="https://img.shields.io/badge/crowdin%20%28projects%29-button?style=for-the-badge&logo=crowdin&color=gray" alt="Crowdin (projects)">
  </a>
  <a href="https://crowdin.com/project/lizardbyte-docs">
    <img src="https://img.shields.io/badge/crowdin%20%28docs%29-button?style=for-the-badge&logo=crowdin&color=gray" alt="Crowdin (docs)">
  </a>
  <a href="https://github.com/organizations/LizardByte/settings/actions/caches">
    <img src="https://img.shields.io/badge/github%20caches-button?style=for-the-badge&logo=github-actions&color=gray" alt="GitHub Caches">
  </a>
  <a href="https://github.com/organizations/LizardByte/settings/actions/hosted-runners">
    <img src="https://img.shields.io/badge/github%20hosted%20runners-button?style=for-the-badge&logo=github&color=gray" alt="GitHub Hosted Runners">
  </a>
  <a href="https://pypi.org/user/LizardByte/">
    <img src="https://img.shields.io/badge/pypi-button?style=for-the-badge&logo=pypi&color=gray" alt="PyPI">
  </a>
  <a href="https://readthedocs.org/dashboard/lizardbyte/subprojects/">
    <img src="https://img.shields.io/badge/readthedocs-button?style=for-the-badge&logo=readthedocs&color=gray" alt="ReadTheDocs">
  </a>
  <a href="https://sonarcloud.io/organizations/lizardbyte/projects">
    <img src="https://img.shields.io/badge/sonarcloud-button?style=for-the-badge&logo=sonarcloud&color=gray" alt="SonarCloud">
  </a>
</div>

<div align="center" id="Under Construction">
  <h3>Under Construction</h3>
  <a href="https://cloudsmith.io/~lizardbyte/repos">
    <img src="https://img.shields.io/badge/cloudsmith-button?style=for-the-badge&logo=cloudsmith&color=yellow" alt="Cloudsmith">
  </a>
  <a href="https://copr.fedorainfracloud.org/coprs/lizardbyte">
    <img src="https://img.shields.io/badge/fedora%20copr-button?style=for-the-badge&logo=fedora&color=yellow" alt="copr">
  </a>
  <a href="https://lizardbyte.youtrack.cloud">
    <img src="https://img.shields.io/badge/youtrack-button?style=for-the-badge&logo=jetbrains&color=yellow" alt="YouTrack">
  </a>
</div>

## Repository Data

In [ ]:
# Imports

# standard imports
import json
import os
import time

# lib imports
from dotenv import load_dotenv
from github import Github, UnknownObjectException
from IPython.display import HTML, display
from itables import init_notebook_mode, show
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
# Setup the environment

# Load environment variables from .env file
load_dotenv()

# Authenticate with GitHub
token = os.getenv("GITHUB_TOKEN")
g = Github(token)

# set the default plotly template
pio.templates.default = "plotly_dark"

# Fetch repository data
org_name = "LizardByte"
org = g.get_organization(org_name)
repos = org.get_repos()

# constants
text_template = '%{text}'
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'gh-pages')

# all readthedocs projects
# readthedocs data
readthedocs_path = os.path.join(data_dir, 'readthedocs', 'projects.json')

with open(readthedocs_path, 'r') as f:
    readthedocs_data = json.load(f)

In [ ]:
# Get Repo Data
repo_data = []
for repo in repos:
    # get license
    license_name = repo.license.name if repo.license else "No License"

    # split open issues and PRs
    open_issues = repo.get_issues(state='open')
    open_prs = [issue for issue in open_issues if issue.pull_request is not None]
    open_issues = [issue for issue in open_issues if issue.pull_request is None]

    # coverage data
    coverage = 0
    try:
        with open(os.path.join(data_dir, 'codecov', f'{repo.name}.json')) as f:
            coverage_data = json.load(f)
        coverage = coverage_data['totals']['coverage']
    except Exception:
        pass

    # readthedocs data
    readthedocs_project = None
    for project in readthedocs_data:
        if project['repository']['url'] == repo.clone_url:
            readthedocs_project = project

    # has README.md or README.rst
    # check if the repo has a README.md or README.rst
    readme_file = None
    try:
        readme_file = repo.get_readme()
    except UnknownObjectException:
        pass

    repo_data.append({
        "repo": repo.name,
        "stars": repo.stargazers_count,
        "archived": repo.archived,
        "fork": repo.fork,
        "forks": repo.forks_count,
        "issues": open_issues,
        "topics": repo.get_topics(),
        "languages": repo.get_languages(),
        "license": license_name,
        "prs": open_prs,
        "created_at": repo.created_at,
        "updated_at": repo.updated_at,
        "coverage": coverage,
        "readthedocs": readthedocs_project,
        "has_readthedocs": readthedocs_project is not None,
        "has_readme": readme_file is not None,
        "_repo": repo,
    })

In [ ]:
# Initial data frames
df = pd.DataFrame(repo_data)
df_repos = df[
    (~df['archived']) &
    (~df['topics'].apply(lambda topics: 'package-manager' in topics))
]
df_original_repos = df[
    (~df['archived']) &
    (~df['fork']) &
    (~df['topics'].apply(lambda topics: 'package-manager' in topics))
]

In [ ]:
# Initial Results
print(f'Last Updated: {time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())} UTC')
print(f'Total Repositories: {len(repo_data)}')
print(f'Archived Repositories: {df["archived"].sum()}')
print(f'Forked Repositories: {df["fork"].sum()}')
print(f'Total Open Issues: {df["issues"].apply(len).sum()}')
print(f'Total Open PRs: {df["prs"].apply(len).sum()}')
print(f'Open issues in active repositories: {df_repos["issues"].apply(len).sum()}')
print(f'Open PRs in active repositories: {df_repos["prs"].apply(len).sum()}')

### Star Gazers

In [ ]:
# Stars
df_stars = df_repos.sort_values(
    by='stars',
    ascending=False,
)
df_stars['log_stars'] = np.log1p(df_stars['stars'])
fig = px.bar(
    df_stars,
    x='repo',
    y='log_stars',
    title='Stars',
    text='stars',
)
fig.update_traces(
    texttemplate=text_template,
    textposition='inside',
)
fig.update_layout(
    yaxis_title=None,
    yaxis_showticklabels=False,
)
fig.show()

In [ ]:
# Star History Data
stargazer_data = []
for repo in df_repos.to_dict('records'):
    stargazers = repo['_repo'].get_stargazers_with_dates()
    for stargazer in stargazers:
        stargazer_data.append({
            "repo": repo['repo'],
            "date": stargazer.starred_at,
        })

In [ ]:
# Star History Visuals
df_stargazers = pd.DataFrame(stargazer_data)
df_stargazers = df_stargazers.sort_values(by="date")
df_stargazers["cumulative_stars"] = df_stargazers.groupby("repo").cumcount() + 1

fig = px.line(
    df_stargazers,
    x="date",
    y="cumulative_stars",
    color="repo",
    title="Star History",
    labels={"date": "Date", "cumulative_stars": "Cumulative Stars"},
)
fig.show()

### Forks

In [ ]:
# Forks
df_forks = df_repos.sort_values(
    by='forks',
    ascending=False,
)
df_forks['log_forks'] = np.log1p(df_forks['forks'])
fig = px.bar(
    df_forks,
    x='repo',
    y='log_forks',
    title='Forks',
    text='forks',
)
fig.update_traces(
    texttemplate=text_template,
    textposition='inside',
)
fig.update_layout(
    yaxis_title=None,
    yaxis_showticklabels=False,
)
fig.show()

### Open Issues

In [ ]:
# Open Issues
df_issues = df_repos.copy()
df_issues['issue_count'] = df_issues['issues'].apply(len)
df_issues = df_issues.sort_values(by='issue_count', ascending=False)
df_issues['log_issues'] = np.log1p(df_issues['issue_count'])
fig = px.bar(
    df_issues,
    x='repo',
    y='log_issues',
    title='Open Issues',
    text='issue_count',
)
fig.update_traces(
    texttemplate=text_template,
    textposition='inside',
)
fig.update_layout(
    yaxis_title=None,
    yaxis_showticklabels=False,
)
fig.show()

### Open PRs

In [ ]:
# PR Data
pr_data = []
for repo in df_repos.to_dict('records'):
    for pr in repo['prs']:
        pr_details = repo['_repo'].get_pull(pr.number)

        # Check if the PR has been approved
        reviews = pr_details.get_reviews()
        approved = any(review.state == 'APPROVED' for review in reviews)

        # Get the milestone
        milestone = pr_details.milestone.title if pr_details.milestone else None

        pr_data.append({
            "repo": repo['repo'],
            "number": pr_details.number,
            "title": pr_details.title,
            "author": pr_details.user.login,
            "labels": [label.name for label in pr_details.labels],
            "assignees": [assignee.login for assignee in pr_details.assignees],
            "created_at": pr_details.created_at,
            "last_activity": pr_details.updated_at,
            "status": "Draft" if pr_details.draft else "Ready",
            "approved": approved,
            "milestone": milestone,
        })

In [ ]:
# Create DataFrame for PR details
df_pr_details = pd.DataFrame(pr_data)

# Group by repository and status to get the count of PRs
df_pr_counts = df_pr_details.groupby(['repo', 'status']).size().reset_index(name='pr_count')

# Sort repositories by total PR count
df_pr_counts['total_prs'] = df_pr_counts.groupby('repo')['pr_count'].transform('sum')
df_pr_counts = df_pr_counts.sort_values(by='total_prs', ascending=False)

# Create Stacked Bar Chart
fig_bar = px.bar(
    df_pr_counts,
    x='repo',
    y='pr_count',
    color='status',
    title='Open Pull Requests',
    labels={'pr_count': 'Count of PRs', 'repo': 'Repository', 'status': 'PR Status'},
    text='pr_count',
    category_orders={'repo': df_pr_counts['repo'].tolist()},
)

fig_bar.update_layout(
    yaxis_title='Open PRs',
    xaxis_title='Repository',
)
fig_bar.update_traces(
    texttemplate=text_template,
    textposition='inside',
)
fig_bar.show()

In [ ]:
# PR Table

# darken the column filter inputs
css = """
.dt-column-title input[type="text"] {
  background-color: var(--jp-layout-color0);
  border-color: rgb(64,67,70);
  border-width: 1px;
  color: var(--jp-ui-font-color1);
}
"""
display(HTML(f"<style>{css}</style>"))

init_notebook_mode(
    all_interactive=True,
    connected=False,
)

# Display the DataFrame as an interactive table using itables
table_download_name = "LizardByte-Pull-Requests"
show(
    df_pr_details,
    buttons=[
        "pageLength",
        "copyHtml5",
        {"extend": "csvHtml5", "title": table_download_name},
        {"extend": "excelHtml5", "title": table_download_name},
    ],
    classes="display compact",
    column_filters="header",
    header=True,
    layout={"topEnd": None},
)

### License Distribution

In [ ]:
# License distribution
license_counts = df_repos.groupby(['license', 'repo']).size().reset_index(name='count')

fig_treemap = px.treemap(
    license_counts,
    path=['license', 'repo'],
    values='count',
    title='License Distribution',
    hover_data={'repo': True, 'count': False},
)
fig_treemap.show()

### Coverage

In [ ]:
# Coverage
df_coverage = df_repos.sort_values(
    by='coverage',
    ascending=False,
)

# inverse marker size, so higher coverage has smaller markers
df_coverage['marker_size'] = df_coverage['coverage'].apply(lambda x: 110 - x if x > 0 else 0)

fig_scatter = px.scatter(
    df_coverage,
    x='repo',
    y='coverage',
    title='Coverage Percentage',
    size='marker_size',
    color='coverage',
    color_continuous_scale=['red', 'yellow', 'green'],  # red is low, green is high
)
fig_scatter.update_layout(
    yaxis_title='Coverage Percentage',
    xaxis_title='Repository',
)
fig_scatter.show()

### Programming Languages

In [ ]:
# Programming language data
language_data = []
for repo in df_repos.to_dict('records'):
    for language, bytes_of_code in repo['languages'].items():
        language_data.append({
            "repo": repo['repo'],
            "language": language,
            "bytes_of_code": bytes_of_code,
        })

In [ ]:
# Programming Languages
df_languages = pd.DataFrame(language_data)

# Aggregate data by language and repo
language_counts_bytes = df_languages.groupby(['language', 'repo']).agg({
    'bytes_of_code': 'sum'
}).reset_index()
language_counts_repos = df_languages.groupby(['language', 'repo']).size().reset_index(name='repo_count')

def create_language_figures(counts: pd.DataFrame, path_key: str, value_key: str):
    _fig_treemap = px.treemap(
        counts,
        path=[path_key, 'repo'],
        values=value_key,
    )
    _fig_sunburst = px.sunburst(
        counts,
        path=[path_key, 'repo'],
        values=value_key,
    )
    return _fig_treemap, _fig_sunburst

# List of tuples containing the data and titles for each figure
figures_data = [
    (language_counts_bytes, 'language', 'bytes_of_code', 'Programming Languages by Bytes of Code'),
    (language_counts_repos, 'language', 'repo_count', 'Programming Languages by Repo Count')
]

# Loop through the list to create figures and add traces
for _counts, _path_key, value_key, title in figures_data:
    fig_treemap, fig_sunburst = create_language_figures(counts=_counts, path_key=_path_key, value_key=value_key)

    fig = go.Figure()
    fig.add_trace(fig_treemap.data[0])
    fig.add_trace(fig_sunburst.data[0])
    fig.data[1].visible = False

    fig.update_layout(
        title=title,
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": "Treemap",
                        "method": "update",
                        "args": [
                            {"visible": [True, False]},
                        ],
                    },
                    {
                        "label": "Sunburst",
                        "method": "update",
                        "args": [
                            {"visible": [False, True]},
                        ],
                    },
                ],
                "direction": "down",
                "showactive": True,
            }
        ]
    )
    fig.show()

### Documentation

In [ ]:
# Docs data
docs_data = []
for repo in df_repos.to_dict('records'):
    docs_data.append({
        "repo": repo['repo'],
        "has_readme": repo['has_readme'],
        "has_readthedocs": repo['has_readthedocs'],
    })

In [ ]:
# Docs
df_docs = pd.DataFrame(docs_data)
readme_counts = df_docs.groupby(['has_readme', 'repo']).size().reset_index(name='repo_count')
readthedocs_counts = df_docs.groupby(['has_readthedocs', 'repo']).size().reset_index(name='repo_count')

def create_figures(counts: pd.DataFrame, path_key: str):
    _fig_treemap = px.treemap(
        counts,
        path=[path_key, 'repo'],
        values='repo_count',
    )
    _fig_sunburst = px.sunburst(
        counts,
        path=[path_key, 'repo'],
        values='repo_count',
    )
    return _fig_treemap, _fig_sunburst

# List of tuples containing the data and titles for each figure
figures_data = [
    (readme_counts, 'has_readme', 'Has README file'),
    (readthedocs_counts, 'has_readthedocs', 'Uses ReadTheDocs')
]

# Loop through the list to create figures and add traces
for _counts, _path_key, title in figures_data:
    fig_treemap, fig_sunburst = create_figures(counts=_counts, path_key=_path_key)

    fig = go.Figure()
    fig.add_trace(fig_treemap.data[0])
    fig.add_trace(fig_sunburst.data[0])
    fig.data[1].visible = False

    fig.update_layout(
        title=title,
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": "Treemap",
                        "method": "update",
                        "args": [{"visible": [True, False]}],
                    },
                    {
                        "label": "Sunburst",
                        "method": "update",
                        "args": [{"visible": [False, True]}],
                    },
                ],
                "direction": "down",
                "showactive": True,
            }
        ]
    )
    fig.show()